In [1]:
import torch
import Models
from transformers import BertForSequenceClassification


nb_bert_pipe = Models.get_nb_bert(2, model_type=BertForSequenceClassification)
mbert_pipe = Models.get_mbert(2, model_type=BertForSequenceClassification)
nor_bert_pipe = Models.get_nor_bert(2, model_type=BertForSequenceClassification)


Some weights of the model checkpoint at NbAiLab/nb-bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initi

In [2]:
import json

def number_sentiment(sentiment):
    if sentiment == 'Positive':
        return 1
    return 0

with open('./sentiment_polarity/train.json') as polarity_data:
    polarity_array = json.load(polarity_data)[:300]
    train_texts = [datapoint['text'] for datapoint in polarity_array]
    train_labels = [number_sentiment(datapoint['label']) for datapoint in polarity_array]
    
with open('./sentiment_polarity/val.json') as polarity_data:
    polarity_array = json.load(polarity_data)[:300]
    val_texts = [datapoint['text'] for datapoint in polarity_array]
    val_labels = [number_sentiment(datapoint['label']) for datapoint in polarity_array]
    
with open('./sentiment_polarity/test.json') as polarity_data:
    polarity_array = json.load(polarity_data)
    test_texts = [datapoint['text'] for datapoint in polarity_array]
    test_labels = [number_sentiment(datapoint['label']) for datapoint in polarity_array]

In [3]:
from transformers import AutoTokenizer
nor_bert_tokenizer = AutoTokenizer.from_pretrained("ltgoslo/norbert")
nb_bert_tokenizer = AutoTokenizer.from_pretrained('NbAiLab/nb-bert-base')
mbert_tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')


nor_bert_train_encodings = nor_bert_tokenizer(train_texts, truncation=True, padding=True)
nb_bert_train_encodings = nb_bert_tokenizer(train_texts, truncation=True, padding=True)
mbert_train_encodings = mbert_tokenizer(train_texts, truncation=True, padding=True)

nor_bert_val_encodings = nor_bert_tokenizer(val_texts, truncation=True, padding=True)
nb_bert_val_encodings = nb_bert_tokenizer(val_texts, truncation=True, padding=True)
mbert_val_encodings = mbert_tokenizer(val_texts, truncation=True, padding=True)

nor_bert_test_encodings = nor_bert_tokenizer(test_texts, truncation=True, padding=True)
nb_bert_test_encodings = nb_bert_tokenizer(test_texts, truncation=True, padding=True)
mbert_test_encodings = mbert_tokenizer(test_texts, truncation=True, padding=True)


class SentinentPolarityDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


nor_bert_train_dataset = SentinentPolarityDataset(nor_bert_train_encodings, train_labels)
nb_bert_train_dataset = SentinentPolarityDataset(nb_bert_train_encodings, train_labels)
mbert_train_dataset = SentinentPolarityDataset(mbert_train_encodings, train_labels)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [4]:
from torch.utils.data import DataLoader
from transformers import AdamW
from tqdm import tqdm

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model=nor_bert_pipe.model
model.to(device)
model.train()

train_loader = DataLoader(nor_bert_train_dataset, batch_size=4, shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)

for epoch in range(3):
    for batch in tqdm(train_loader):
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optim.step()

model.eval()

100%|██████████| 75/75 [03:37<00:00,  2.91s/it]


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32922, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [5]:
# Evaluate using test set
nor_bert_test_dataset = SentinentPolarityDataset(nor_bert_test_encodings, train_labels)

In [6]:
# Do not change the fucking batch size
test_loader = DataLoader(nor_bert_test_dataset, batch_size=1, shuffle=True)
TP = 0
TN = 0
FP = 0
FN = 0
for batch in tqdm(test_loader):
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)
    outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
    if(outputs['logits'][0][0] > outputs['logits'][0][1]):
        # False prediction
        if labels[0] == 1:
            FN += 1
        else:
            TN += 1
    else:
        # True prediction
        if labels[0] == 1:
            TP += 1
        else:
            FP += 1

100%|██████████| 300/300 [01:10<00:00,  4.24it/s]


In [10]:
f_score = TP / (TP + (0.5* (FP + FN)))
print('F1-score: ', f_score)

F1-score:  0.8477842003853564


In [11]:
print('Accuracy: ', (TP + TN) / (TP + TN + FP + FN))

Accuracy:  0.7366666666666667


In [9]:
print(FN)

21
